## COG

## Initialise COG

### Load packages

In [1]:
import os
import sys
import html
import requests
import gdal
import rasterio
import geopandas as gpd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from datetime import datetime
from lxml import etree
from tempfile import NamedTemporaryFile

sys.path.append('../../../Scripts')
from dea_dask import create_local_dask_cluster

sys.path.append('../../shared')
import satfetcher

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Set up a dask cluster

In [2]:
# initialise the cluster
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:43937 Dashboard: /user/lewis/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.11 GB


## Get study area polygon

### Load study area geometry as geojson

In [3]:
# read study area and get bounds as a list
gdf = gpd.read_file('../../data/cog/yandisa.geojson')
gdf_bounds = gdf.bounds.values[0].tolist()

## Set STAC Search parameters

In [4]:
# get satellite collection on dea. todo get from user in arcgis, sentinel 2
collections = [
    'ga_ls5t_ard_3', 
    'ga_ls7e_ard_3',
    'ga_ls8c_ard_3'
]

# set required bands
bands = [
    'oa_fmask',
    'nbart_blue', 
    'nbart_green', 
    'nbart_red', 
    'nbart_nir',
    'nbart_swir_1',
    'nbart_swir_2'
]

# get satellite collection date range, convert to stac. todo get from user in arcgis
start_dt, end_dt = '1990-01-01', '1995-12-31'

# bring it all together for a query
query = {
    'collections': collections,
    'datetime': '{0}/{1}'.format(start_dt, end_dt),
    'bbox': gdf_bounds,
    'query': {'eo:cloud_cover': {'lt': 5}}, #this doesnt work
    'limit': 1000
}

## Fetch DEA Public Data via STAC Search

In [5]:
# set stac endpoint
search_endpoint = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'

# send and get request for stac json using 
stac_response = requests.post(search_endpoint, json=query)

# check for response empty errors, convert to json if so
if stac_response.ok:
    stac_response = stac_response.json()
    num_items = len(stac_response.get('features'))
    print('Found {0} satellite scenes in total.'.format(num_items))
else:
    raise ValueError('Could not connect to DEA STAC SEARCH endpoint.')

Found 146 satellite scenes in total.


## Iterate STAC response and remove cloud cover

In [6]:
# set max cloud cover (0 - 100)
max_cloud = 50

# get num of all stac scenes
num_all_items = len(stac_response.get('features'))

feat_list = []
for feat in stac_response.get('features'):
    if max_cloud > float(feat.get('properties').get('eo:cloud_cover')):
        feat_list.append(feat)
        
# count cloud less scenes and compare
if feat_list:
    num_clean_items = len(feat_list)
    print('Removed {0} satellite scenes due to clouds.'.format(num_all_items - num_clean_items))
    print('Total of {0} satellite scenes remaining.'.format(num_clean_items))

Removed 10 satellite scenes due to clouds.
Total of 136 satellite scenes remaining.


## Set VRT Dataset and Raster generators

In [215]:
# meta, checks
def make_vrt_dataset_xml(x_size, y_size, axis_map, srs, trans):
    """
    """
    
    # imports
    from lxml import etree as et

    # set up root vrt dataset elem
    xml_ds = '<VRTDataset rasterXSize="{x_size}" rasterYSize="{y_size}"></VRTDataset>'
    xml_ds = et.fromstring(xml_ds.format(x_size=x_size, y_size=y_size))

    # set up srs element and add to vrt dataset
    xml_srs = '<SRS dataAxisToSRSAxisMapping="{axis_map}">{srs}</SRS>'
    xml_ds.append(et.fromstring(xml_srs.format(axis_map=axis_map, srs=srs)))

    # set up geo transform element and add to vrt dataset
    xml_trans = '<GeoTransform>{trans}</GeoTransform>'
    xml_ds.append(et.fromstring(xml_trans.format(trans=trans)))
    
    # return xml dataset
    return xml_ds

# meta, checks
def make_vrt_raster_xml(x_size, y_size, dtype, band_num, nodata, dt, rel_to_vrt, url, src_band):
    """
    """

    # imports
    from lxml import etree as et
    
    # set up root vrt raster elem
    xml_rast = '<VRTRasterBand dataType="{dtype}" band="{band_num}"></VRTRasterBand>'
    xml_rast = et.fromstring(xml_rast.format(dtype=dtype, band_num=band_num))
        
    # add a top-level nodata value element and add to vrt raster
    xml_ndv = '<NoDataValue>{nodata}</NoDataValue>'
    xml_rast.append(et.fromstring(xml_ndv.format(nodata=nodata)))    
    
    # set up top-level complexsource element, dont add it to rast yet
    xml_complex = '<ComplexSource></ComplexSource>'
    xml_complex = et.fromstring(xml_complex)
    
    # add a description elem to hold datetime to the complex source
    xml_desc = '<Description>{dt}</Description>'
    xml_complex.append(et.fromstring(xml_desc.format(dt=dt)))

    # add source filename to complex source
    xml_filename = '<SourceFilename relativeToVRT="{rel_to_vrt}">/vsicurl/{url}</SourceFilename>'
    xml_complex.append(et.fromstring(xml_filename.format(rel_to_vrt=rel_to_vrt, url=url)))
    
    # add source band num to complex source
    xml_src_band = '<SourceBand>{src_band}</SourceBand>'
    xml_complex.append(et.fromstring(xml_src_band.format(src_band=src_band)))
    
    # add source properties to complex source. hardcoded block size for now
    xml_src_props = '<SourceProperties RasterXSize="{x_size}" RasterYSize="{y_size}"' + ' ' + \
                    'DataType="{dtype}" BlockXSize="512" BlockYSize="512"></SourceProperties>'
    xml_complex.append(et.fromstring(xml_src_props.format(x_size=x_size, y_size=y_size, dtype=dtype)))
    
    # add a src rect to complex source. hardcoded offset for now
    xml_src_rect = '<SrcRect xOff="0" yOff="0" xSize="{x_size}" ySize="{y_size}"></SrcRect>'
    xml_complex.append(et.fromstring(xml_src_rect.format(x_size=x_size, y_size=y_size)))
    
    # add a dst rect to complex source. hardedcoded offset for now
    xml_dst_rect = '<DstRect xOff="0" yOff="0" xSize="{x_size}" ySize="{y_size}"></DstRect>'
    xml_complex.append(et.fromstring(xml_dst_rect.format(x_size=x_size, y_size=y_size)))
    
    # add a lower-level nodata elem to complex source
    xml_nd = '<NODATA>{nodata}</NODATA>'
    xml_complex.append(et.fromstring(xml_nd.format(nodata=nodata)))
        
    # finally, add filled in complex source element to rast
    xml_rast.append(xml_complex)
    
    # return xml raster
    return xml_rast

In [248]:
xml_ds = make_vrt_dataset_xml(x_size=7871,
                              y_size=7061,
                              axis_map='1,2',
                              srs='PROJCS[...]',
                              trans='30, 0, 30, 1')

xml_rast = make_vrt_raster_xml(x_size=7871,
                               y_size=7061,
                               dtype='Int16',
                               band_num=1,
                               nodata=-999,
                               dt='2019-01-01T00:00:01',
                               rel_to_vrt=0,
                               url='https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band01.tif',
                               src_band=1                               
                              )

xml_ds.append(xml_rast)

from lxml import etree as et
out = et.tostring(xml_ds)

In [ ]:
def build_vrt_raster(x_size, y_size, dtype, dt, url, relative_to, band_num, nodata):
    """
    """
    
    # check types
    
    # create vrt dataset xml element
    xml_rast = etree.Element('VRTRasterBand', 
                             dataType="{}".format(dtype),
                             band="{}".format(band_num))
    
    # create nodata xml element
    xml_nodata = etree.Element('NoDataValue')
    xml_nodata.text = '{}'.format(nodata)
    xml_rast.append(xml_nodata)
    
    # create raster description xml element
    xml_desc = etree.Element('Description')
    xml_desc.text = '{}'.format(dt)
    xml_rast.append(xml_desc)
      
    # create simple source xml element
    xml_complex = etree.Element('ComplexSource')

    # create SourceFilename xml element
    xml_filename = etree.Element('SourceFilename',
                                 relativeToVRT="{}".format(relative_to))
    xml_filename.text = '/vsicurl/{}'.format(url)
    xml_complex.append(xml_filename)

    # create simple source xml element
    xml_band = etree.Element('SourceBand')
    xml_band.text = '{}'.format(band_num)  # was 1, as official vrtbuilder does it, but this helps id
    xml_complex.append(xml_band)
    
    # create SourceProperties xml element
    xml_props = etree.Element('SourceProperties',
                              rasterXSize="{}".format(x_size),
                              rasterYSize="{}".format(y_size),
                              DataType="{}".format(dtype),
                              BlockXSize="{}".format(512),
                              BlockYSize="{}".format(512))
    xml_complex.append(xml_props)

    # create source rect xml element
    xml_src_rect = etree.Element('SrcRect',
                                 xOff="{}".format(0),
                                 yOff="{}".format(0),
                                 xSize="{}".format(x_size),
                                 ySize="{}".format(y_size))
    xml_complex.append(xml_src_rect)
    
    # create destination rect xml element
    xml_dst_rect = etree.Element('DstRect',
                                 xOff="{}".format(0),
                                 yOff="{}".format(0),
                                 xSize="{}".format(x_size),
                                 ySize="{}".format(y_size))
    xml_complex.append(xml_dst_rect)
    
    # create nodata xml element
    xml_src_nodata = etree.Element('NODATA')
    xml_src_nodata.text = '{}'.format(nodata)
    xml_complex.append(xml_src_nodata)
        
    # add simple element to raster element as child
    xml_rast.append(xml_complex)
    
    return xml_rast

## Fill VRT temmplate with values

In [ ]:
# todo checks, meta
def generate_vrt(feat_list, band=None):
    """
    band : list, str
        Can be a list or string of name of band(s) required.
    """
        
    # check if band provided, if so and is str, make list
    if band is None:
        bands = []
    elif not isinstance(band, list):
        bands = [band]
    else:
        bands = band
        
    # check if bands in list
    allowed_bands = [
        'nbart_blue', 
        'nbart_green',
        'nbart_red',
        'nbart_nir',
        'nbart_swir_1',
        'nbart_swir_2',
        'oa_mask'
    ]
    
    # ensure requested bands allowed
    for b in bands:
        if b not in allowed_bands:
            raise ValueError('Requested an unsupported band.')
            
    # check features type, length
    if not isinstance(feat_list, list):
        raise TypeError('Features must be a list of xml objects.')
    elif not len(feat_list) > 0:
        raise ValueError('No features provided.')

    # set list vrt of each scene
    vrt_list = []

    # iter stac scenes, get metadata, insert bands into vrt template
    for feat_idx, feat in enumerate(feat_list, start=1):

        # get scene identity and properties
        f_id = feat.get('id')
        f_props = feat.get('properties')

        # get scene-level date
        f_dt = f_props.get('datetime')

        # get scene-level x, y parameters
        f_x_size = f_props.get('proj:shape')[1]
        f_y_size = f_props.get('proj:shape')[0]

        # get scene-level epsg src as wkt
        f_srs = rasterio.crs.CRS.from_epsg(f_props.get('proj:epsg')).wkt

        # get scene-level transform
        aff = rasterio.transform.Affine(*f_props.get('proj:transform')[0:6])
        f_transform = ', '.join(str(p) for p in rasterio.transform.Affine.to_gdal(aff))

        # generate vrt dataset 
        vrt_xml = build_vrt_dataset(x_size=f_x_size, 
                                    y_size=f_y_size, 
                                    srs=f_srs, 
                                    transform=f_transform)

        # iterate bands and add to vrt if exists
        band_idx = 1
        for band in bands:
            if band in feat.get('assets'):

                # get asset
                asset = feat.get('assets').get(band)

                # set dtype to in16 unless mask
                a_dtype = 'Int8' if band == 'oa_mask' else 'Int16'

                # get asset raster x, y sizes
                a_x_size = asset.get('proj:shape')[1]
                a_y_size = asset.get('proj:shape')[0]

                # get raster url, replace s3 with https
                a_url = asset.get('href')
                a_url = a_url.replace('s3://dea-public-data', 'https://data.dea.ga.gov.au')

                # get epsg as wkt
                #srs = rasterio.crs.CRS.from_epsg(f_epsg).wkt

                # get transform (six params) as string
                aff = rasterio.transform.Affine(*asset.get('proj:transform')[0:6])
                a_transform = ', '.join(str(p) for p in rasterio.transform.Affine.to_gdal(aff))

                # get nodata value
                a_nodata = -999

                # build raster xml
                rast_xml = build_vrt_raster(x_size=a_x_size, 
                                            y_size=a_y_size, 
                                            dtype=a_dtype, 
                                            dt=f_dt,
                                            url=a_url, 
                                            relative_to=1, 
                                            band_num=band_idx, 
                                            nodata=a_nodata)

                # add raster xml to vrt dataset xml as child
                vrt_xml.append(rast_xml)

                # increase band index
                band_idx += 1

        # decode to utf-8 string and append to vrt list
        vrt_str = etree.tostring(vrt_xml).decode('utf-8')
        vrt_list.append(vrt_str)
        
    return vrt_list

In [ ]:
# create list of bands needed
wanted_bands = [
    'nbart_blue', 
    'nbart_green',
    'nbart_red',
    'nbart_nir',
    'nbart_swir_1',
    'nbart_swir_1',
    'oa_mask'
]

# build datetimes and vrts for each band
#for band in wanted_bands:

# todo, iterate this via list above
vrt_blue = generate_vrt(feat_list=feat_list, band='nbart_blue')
vrt_green = generate_vrt(feat_list=feat_list, band='nbart_green')
vrt_red = generate_vrt(feat_list=feat_list, band='nbart_red')
vrt_nir = generate_vrt(feat_list=feat_list, band='nbart_nir')
vrt_swir_1 = generate_vrt(feat_list=feat_list, band='nbart_swir_1')
vrt_swir_2 = generate_vrt(feat_list=feat_list, band='nbart_swir_2')
vrt_mask = generate_vrt(feat_list=feat_list, band='oa_mask')

## Build an completed in-memory VRT file

In [ ]:
# checks, meta
def create_vrt_file(vrt_files):
    """
    """
    
    # checks
    
    # load up a temp named file and create vrt
    with NamedTemporaryFile() as tmp:

        # set vrt options
        vrt_opts = gdal.BuildVRTOptions(separate=True,
                                        #bandList=[1],
                                        #outputBounds=boundingbox,
                                        #resampleAlg='bilinear',
                                        #resolution='user',
                                        #xRes=30.0,
                                        #yRes=30.0,
                                        #outputSRS=rasterio.crs.CRS.from_epsg(3577).wkt
                                        #targetAlignedPixels=True
                                       )
        
        # build vrt, close it (to create it)
        vrt_out = gdal.BuildVRT(tmp.name, vrt_files, options=vrt_opts)
        vrt_out = None

        # warp and translate funcs
        # todo: MAY NEED

        # read it in to memory and decode it
        vrt = tmp.read().decode("utf-8")
        return vrt

In [ ]:
# small subset of raster in utm 50N
#bb = [683100.0, -2542470.0, 686070.0, -2539500.0]

# todo improve this code

# create vrts
vrt_blue_out = create_vrt_file(vrt_files=vrt_blue)
vrt_green_out = create_vrt_file(vrt_files=vrt_green)
vrt_red_out = create_vrt_file(vrt_files=vrt_red)
vrt_nir_out = create_vrt_file(vrt_files=vrt_nir)
vrt_swir_1_out = create_vrt_file(vrt_files=vrt_swir_1)
vrt_swir_2_out = create_vrt_file(vrt_files=vrt_swir_2)
vrt_mask_out = create_vrt_file(vrt_files=vrt_mask)

## Parse datetime strings into map

In [ ]:
def parse_datetimes(vrt_string):
    
    # convert html tags back out
    clean_elem = html.unescape(vrt_blue_out)

    # convert string to etree elements
    root = etree.fromstring(clean_elem)

    # pull descriptions out to get date times
    elem_desc = root.findall('.//Description')

    # iterate elements and pull description text
    dt_map = {}
    for i, e in enumerate(elem_desc, start=1):
        dt_map[i] = e.text
        
    return dt_map

In [ ]:
# create vrts
dt_blue = parse_datetimes(vrt_string=vrt_blue_out)
dt_green = parse_datetimes(vrt_string=vrt_green_out)
dt_red = parse_datetimes(vrt_string=vrt_red_out)
dt_nir = parse_datetimes(vrt_string=vrt_nir_out)
dt_swir_1 = parse_datetimes(vrt_string=vrt_swir_1_out)
dt_swir_2 = parse_datetimes(vrt_string=vrt_swir_2_out)
dt_mask = parse_datetimes(vrt_string=vrt_mask_out)

# check if lengths are all same


## Convert to chunked dataset

In [ ]:
def build_xr_dataset(vrt_file, band_name):
    
    # setup chunks
    chunks = {'band': 1, 'x': 'auto', 'y': 'auto'}
    
    # load xr as data array
    ds = xr.open_rasterio(vrt_file, chunks=chunks)
    
    # rename default band label to time
    ds = ds.rename({'band': 'time'})
    
    # convert to dataset
    ds = ds.to_dataset(name=band_name, promote_attrs=True)
    
    # subset to coords, bb todo fix this up
    ds = ds.isel(x=slice(4000, 5000), y=slice(3000, 4000))
    
    return ds

In [ ]:
# create datasets
ds_blue = build_xr_dataset(vrt_file=vrt_blue_out, band_name='nbart_blue')

In [ ]:
# replace datetime
def replace_datetimes(ds, dt):
    
    # replace timezone and convert numpy
    dt_dict = {}
    for k, v in dt_blue.items():
        dt_dict[k] = np.datetime64(v.replace('Z', ''))
    
    # remap
    ds['time'] = [dt_dict[i] for i in ds['time'].values.tolist()]
    return ds.sortby('time')
    
ds_blue = replace_datetimes(ds_blue, dt_blue)

In [ ]:
# compute
%time ds_blue = ds_blue.compute()
ds_blue

In [ ]:
# combine all dask datasets into one
#xr.merge([ds_blue, ds_green, ds_red])

## Test download times

### Try raw, without dask

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute()

### Try raw, without dask but with threading

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute(scheduler='threads')

### Try raw, without dask but with processes

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute(scheduler='processes')

### Try dask, with distributed scheduler

In [ ]:
import dask
from dask.distributed import Client
client = Client(processes=True)
client

In [ ]:
# about 47 secs with processes=false, 21 secs when True
%time ds = ds.compute()

### Try dask data arrays split and futures used

In [ ]:
import concurrent.futures 

# create compute func
def compute_da(da):
    return da.compute()

In [ ]:
# split ds into seperate das
da_list = []
for dt in ds['time']:
    da = ds.sel(time=dt)
    da_list.append(da)
    
# try parallel load of all bands
num_cores = 2
with concurrent.futures.ThreadPoolExecutor(num_cores) as executor:
    %time da_list = list(executor.map(compute_da, da_list))
    
ds = xr.concat(da_list, dim='time')

## Working

## Use this to auto gen vrt to test

In [150]:
# really good test env
vrt_1_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band03.tif']
vrt1 = gdal.BuildVRT('vrt_1.vrt', vrt_1_urls, separate=True).FlushCache()

 
vrt_2_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band03.tif']
vrt2 = gdal.BuildVRT('vrt_2.vrt', vrt_2_urls, separate=True).FlushCache()


vrt_3_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band03.tif']
vrt3 = gdal.BuildVRT('vrt_3.vrt', vrt_3_urls, separate=True).FlushCache()


vrt_4_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band03.tif']
vrt4 = gdal.BuildVRT('vrt_4.vrt', vrt_4_urls, separate=True).FlushCache()

# add to list
vrt_list = ['vrt_1.vrt', 'vrt_2.vrt', 'vrt_3.vrt', 'vrt_4.vrt']
vrt_out = gdal.BuildVRT('vrt_all.vrt', vrt_list, separate=False, bandList=[1]).FlushCache()


# read it in to memory and decode it
#vrt_all = tmp.read().decode("utf-8")

# setup chunks
chunks = {'band': 1, 'x': 'auto', 'y': 'auto'}
ds = xr.open_rasterio('vrt_all.vrt', chunks=chunks)
ds = ds.isel(x=slice(2500, 3000), y=slice(2500, 3000))
ds.compute()

IndexError: band index 2 out of range (not in (1,))